In [1]:
import pickle
import numpy as np
import codecs
import random

from nltk.tokenize import wordpunct_tokenize

In [2]:
# load array of objects, where object - class Leaflet
with open("/home/ruslan_yermakov/nlg-ra/datasets/LEAFLET_TRAIN_DATASET.pickle", "rb") as f:
    train_dataset = pickle.load(f)

In [3]:
# load array of objects, where object - class Leaflet
with open("/home/ruslan_yermakov/nlg-ra/datasets/LEAFLET_VALID_DATASET.pickle", "rb") as f:
    valid_dataset = pickle.load(f)

In [4]:
# load array of objects, where object - class Leaflet
with open("/home/ruslan_yermakov/nlg-ra/datasets/LEAFLET_TEST_DATASET.pickle", "rb") as f:
    test_dataset = pickle.load(f)

In [5]:
len(train_dataset)

1068

In [6]:
len(valid_dataset)

134

In [7]:
len(test_dataset)

134

------------------------------------------------------------------------------------------------------

## Produce same output as the script *create_dataset* from data2text-plain

### Plan

section_content -------- true text   

entity_recognition  ------- actually input - set of records   

Set of records in order they appear in section_content    -------- content plan 

In [8]:
RECORD_DELIM = " "
DELIM = u"￨"

HOME = "HOME"
AWAY = "AWAY"

# ENTITY = "Indication"
ENTITY = "Section3"

PAD_WORD = '<blank>'
UNK_WORD = '<unk>'
UNK = 0
BOS_WORD = '<s>'
EOS_WORD = '</s>'

In [9]:
# make sure - sorted entities
def _sort_key(entity):
    return entity['BeginOffset']

def test_order_entities(section_entities):
    
    sorted_entities = sorted(section_entities, key=_sort_key)
    
    if sorted_entities == section_entities: return True
    
    return False


def test_order_dataset(dataset):
    for leaflet in dataset:

        current_leaflet_sections = [leaflet.section1, leaflet.section2, 
                                    leaflet.section3, leaflet.section4, 
                                    leaflet.section5, leaflet.section6]

        for current_section in current_leaflet_sections:

            if current_section.entity_recognition is None:
                continue

            assert test_order_entities(current_section.entity_recognition) == True

            
test_order_dataset(train_dataset)
test_order_dataset(valid_dataset)
test_order_dataset(test_dataset)

In [10]:
def create_summary_contentplan(dataset):
    """
    Transform dataset to be a suitable format for model data2text-plan
    """
    
    # array to store section3 of each leaflet
    summaries_leaflets = []
    
    # array to store content plan of each leaflet
    content_plan_leaflets = []
    
    for leaflet in dataset:
        
        # extract section3 content
        section3_content = leaflet.section3.section_content
        # extract results of NER
        section3_entity_recognition = leaflet.section3.entity_recognition
        
        # skip if either input or output is None
        if section3_content is None or section3_entity_recognition is None:
            continue
        
        # get the content plan of each section
        content_plan_section3 = ''

        for entity in section3_entity_recognition:
            entity_value = entity['Text'] if len(entity['Text'].split(" ")) == 0 else ("_").join(entity['Text'].split(" "))
            entity_type = entity['Type'] if entity['Type'] is not None and len(entity['Type']) > 0 else entity['Category']
            
            # randomly choose HOME or AWAY
            if random.randint(1,2) == 1:
                content_plan_section3 += entity_value + DELIM + ENTITY + DELIM + entity_type + DELIM + HOME
            else:
                content_plan_section3 += entity_value + DELIM + ENTITY + DELIM + entity_type + DELIM + AWAY
            
            if section3_entity_recognition.index(entity) != len(section3_entity_recognition) - 1:
                content_plan_section3 += " "
            else:
                content_plan_section3 += " " + "\n"


        content_plan_leaflets.append(content_plan_section3)

        # get the section3 content
        # make sure to have punctuations as a separate token
        section3_content = wordpunct_tokenize(section3_content)
        
        # back to string
        section3_content = " ".join(section3_content)
        
        # add "\n" at the end
        section3_content = section3_content + "\n"
        
        summaries_leaflets.append(section3_content)
    
    return (content_plan_leaflets, summaries_leaflets)

In [11]:
def add_special_records(records):
    """
    To src_train.txt and src_valid.txt pre-append these special characters, according to data2text-plan project
    """
    
    record = []
    record.append(UNK_WORD)
    record.append(PAD_WORD)
    record.append(PAD_WORD)
    record.append(PAD_WORD)
    records.append(DELIM.join(record))
    record = []
    record.append(PAD_WORD)
    record.append(PAD_WORD)
    record.append(PAD_WORD)
    record.append(PAD_WORD)
    records.append(DELIM.join(record))
    record = []
    record.append(BOS_WORD)
    record.append(PAD_WORD)
    record.append(PAD_WORD)
    record.append(PAD_WORD)
    records.append(DELIM.join(record))
    record = []
    record.append(EOS_WORD)
    record.append(PAD_WORD)
    record.append(PAD_WORD)
    record.append(PAD_WORD)
    records.append(DELIM.join(record))
    
    return records

In [12]:
# get the src_train - training data to be input to the model

def create_src_table(content_plan_leaflets):
    """
    Create src_train - input "table" to the model
    
    Idea: - we do not have a table, so randomized the records in content plan
    
    Update: - do not randomize - make it easier for the model to learn
    """
    
    # store input "table" of each leaflet in array
    src_leaflets = []
    
    for leaflet_content_plan in content_plan_leaflets:
        # remove the end symbol ('\n') of the string
        leaflet_content_plan = leaflet_content_plan[:-2]

        # split string into a list of records
        leaflet_content_plan_collection = leaflet_content_plan.split(" ")
        
        # do not do
        # randomly shuffle records in a list
        # random.shuffle(leaflet_content_plan_collection)
        
        # add special symbols to the begining
        special_symbols = []
        special_symbols = add_special_records(special_symbols)
        
        # create a string containing all the records and special_symbols in the begining
        src_leaflet_section1 = ''

        src_leaflet_section1 += " ".join(special_symbols)
        src_leaflet_section1 += " "

        src_leaflet_section1 += " ".join(leaflet_content_plan_collection)
        src_leaflet_section1 += '\n'


        src_leaflets.append(src_leaflet_section1)
    
    return src_leaflets

In [13]:
def create_training_sets(dataset):
    
    # Output files
    INTER_CONTENT_PLAN = 'inter/train_content_plan.txt'  # intermediate content plan input to second stage
    SRC_FILE = 'src_train.txt'  # src file input to first stage
    TRAIN_TGT_FILE = "tgt_train.txt"  # tgt file of second stage
    CONTENT_PLAN_OUT = 'train_content_plan.txt'  # content plan output of first stage
    
    # Create src - content_plan - summary
    content_plan_leaflets, summaries_leaflets = create_summary_contentplan(dataset)
    src_leaflets = create_src_table(content_plan_leaflets)
    
    # save to corresponding files
    output_file = open(INTER_CONTENT_PLAN, 'w')
    for content_plan in content_plan_leaflets:
        output_file.write(content_plan)
    output_file.close()
    
    summary_file = open(TRAIN_TGT_FILE, 'w')
    for summary_leaflet in summaries_leaflets:
        summary_file.write(summary_leaflet)
    summary_file.close()
    
    src_file = open(SRC_FILE, 'w')
    for src_instance in src_leaflets:
        src_file.write(src_instance)
    src_file.close()
    
    ### create last file needed - e.g (rotowire/train_content_plan.txt)
    inputs = []
    content_plans = []
    with codecs.open(INTER_CONTENT_PLAN, "r", "utf-8") as corpus_file:
        for i, line in enumerate(corpus_file):
            content_plans.append(line.split())

    with codecs.open(SRC_FILE, "r", "utf-8") as corpus_file:
        for i, line in enumerate(corpus_file):
            inputs.append(line.split())
    
    # basically - now content plan POINTs to index in the training dataset
    # content_plan - collection of indexes where each index points to record in training dataset - training_dataset[index]
    
    outputs = []

    for i, training_sample in enumerate(inputs):
        content_plan = content_plans[i]
        output = []
        for record in content_plan:
            output.append(str(training_sample.index(record)))
        outputs.append(" ".join(output))
        
    # write to a file

    output_file = open(CONTENT_PLAN_OUT, 'w')

    # add \n to the end of the string
    output_file.write("\n".join(outputs))
    # add \n between content plans
    output_file.write("\n")

    output_file.close()
    
    return src_leaflets, content_plan_leaflets, summaries_leaflets

In [14]:
leaflets_src_train, leaflets_inter_contentplan_train, leaflets_tgt_train = create_training_sets(train_dataset)

In [15]:
with open('src_train.txt') as reader:
    # This reads the remaining lines from the file object and returns them as a list.
    src_train = reader.readlines()

In [16]:
with open('tgt_train.txt') as reader:
    # This reads the remaining lines from the file object and returns them as a list.
    tgt_train = reader.readlines()

In [17]:
with open('train_content_plan.txt') as reader:
    # This reads the remaining lines from the file object and returns them as a list.
    train_content_plan = reader.readlines()

In [18]:
with open('inter/train_content_plan.txt') as reader:
    # This reads the remaining lines from the file object and returns them as a list.
    inter_train_content_plan = reader.readlines()

======================================================================================================================
## Validation dataset 

In [27]:
def create_validation_sets(dataset):
    
    # Output files    
    INTER_CONTENT_PLAN_VALID = 'inter/valid_content_plan.txt'  # intermediate content plan input to second stage
    SRC_FILE_VALID = 'src_valid.txt'  # src file input to first stage
    TRAIN_TGT_FILE_VALID = "tgt_valid.txt"  # tgt file of second stage
    CONTENT_PLAN_OUT_VALID = 'valid_content_plan.txt'  # content plan output of first stage
    
    # Create src - content_plan - summary
    content_plan_leaflets, summaries_leaflets = create_summary_contentplan(dataset)
    src_leaflets = create_src_table(content_plan_leaflets)
    
    # save to corresponding files
    output_file = open(INTER_CONTENT_PLAN_VALID, 'w')
    for content_plan in content_plan_leaflets:
        output_file.write(content_plan)
    output_file.close()
    
    summary_file = open(TRAIN_TGT_FILE_VALID, 'w')
    for summary_leaflet in summaries_leaflets:
        summary_file.write(summary_leaflet)
    summary_file.close()
    
    src_file = open(SRC_FILE_VALID, 'w')
    for src_instance in src_leaflets:
        src_file.write(src_instance)
    src_file.close()
    
    ### create last file needed - e.g (rotowire/train_content_plan.txt)
    inputs = []
    content_plans = []
    with codecs.open(INTER_CONTENT_PLAN_VALID, "r", "utf-8") as corpus_file:
        for i, line in enumerate(corpus_file):
            content_plans.append(line.split())

    with codecs.open(SRC_FILE_VALID, "r", "utf-8") as corpus_file:
        for i, line in enumerate(corpus_file):
            inputs.append(line.split())
    
    # basically - now content plan POINTs to index in the training dataset
    # content_plan - collection of indexes where each index points to record in training dataset - training_dataset[index]
    
    outputs = []

    for i, training_sample in enumerate(inputs):
        content_plan = content_plans[i]
        output = []
        for record in content_plan:
            output.append(str(training_sample.index(record)))
        outputs.append(" ".join(output))
        
    # write to a file

    output_file = open(CONTENT_PLAN_OUT_VALID, 'w')

    # add \n to the end of the string
    output_file.write("\n".join(outputs))
    # add \n between content plans
    output_file.write("\n")

    output_file.close()
    
    return src_leaflets, content_plan_leaflets, summaries_leaflets

In [28]:
leaflets_src_valid, leaflets_inter_contentplan_valid, leaflets_tgt_valid = create_validation_sets(valid_dataset)

In [29]:
with open('inter/valid_content_plan.txt') as reader:
    # This reads the remaining lines from the file object and returns them as a list.
    contentplan_valid = reader.readlines()

INTER_CONTENT_PLAN_VALID = 'inter/valid_content_plan.txt'  # intermediate content plan input to second stage
    SRC_FILE_VALID = 'src_valid.txt'  # src file input to first stage
    TRAIN_TGT_FILE_VALID = "tgt_valid.txt"  # tgt file of second stage
    CONTENT_PLAN_OUT_VALID = 'valid_content_plan.txt'  # content plan output of first stage

================================================================================================================================
## Test dataset 

In [30]:
def create_test_sets(dataset):
    
    # Output files
    SRC_FILE_TEST = 'test/src_test.txt'  # src file input to first stage
    TRAIN_TGT_FILE_TEST = "test/tgt_test.txt"  # tgt file of second stage 
    
    
    # Create src - content_plan - summary
    content_plan_leaflets, summaries_leaflets = create_summary_contentplan(dataset)
    src_leaflets = create_src_table(content_plan_leaflets)
    
    # save to just summary and src data
    
    summary_file = open(TRAIN_TGT_FILE_TEST, 'w')
    for summary_leaflet in summaries_leaflets:
        summary_file.write(summary_leaflet)
    summary_file.close()
    
    src_file = open(SRC_FILE_TEST, 'w')
    for src_instance in src_leaflets:
        src_file.write(src_instance)
    src_file.close()

    return src_leaflets, summaries_leaflets

In [31]:
leaflets_src_test, leaflets_tgt_test = create_test_sets(test_dataset)

=======================================================================================================================
## Creating *train-roto-ptrs.txt*

In [32]:
# Output files
INTER_CONTENT_PLAN = 'inter/train_content_plan.txt'  # intermediate content plan input to second stage
TRAIN_TGT_FILE = "tgt_train.txt"  # tgt file of second stage
OUTPUT_FILE = "train-roto-ptrs.txt"

In [33]:
with open(TRAIN_TGT_FILE) as reader:
    leaflet_tgt_train = reader.readlines()

In [34]:
with open(INTER_CONTENT_PLAN) as reader:
    leaflets_inter_content_plan = reader.readlines()

For eg: the last entry 245,39 in train_roto_ptrs[1] indicates that the 245th token in summary matches with 39th content plan entry.  

Phoenix ----> Phoenix￨Suns￨TEAM-CITY￨HOME  
Suns ----> Suns￨Suns￨TEAM-NAME￨HOME  
39 ----> 39￨Suns￨TEAM-WINS￨HOME  
38 ----> 38￨Suns￨TEAM-LOSSES￨HOME  
87 ----> 87￨Suns￨TEAM-PTS￨HOME  
85 ----> 85￨Jazz￨TEAM-PTS￨AWAY  
Utah ----> Utah￨Jazz￨TEAM-CITY￨AWAY  

In [35]:
leaflet_tgt_train[0]

"always use cystagon exactly as your doctor or your child ' s doctor has told you . you should check with your doctor if you are not sure . the dose of cystagon prescribed for you or your child will depend on your or your child ' s age and weight . for children up to age 12 years , the dose will be based on the body size ( surface area ), the usual dose being 1 . 30 g / m2 of body surface area per day . for patients over age 12 and over 50 kg weight , the usual dose is 2g / day . in any case the usual dose should not exceed 1 . 95 g / m2 / day . cystagon should be taken or given only by mouth and exactly as your or your child ' s doctor directs . in order for cystagon to work correctly , you must do the following : - follow your doctor ' s directions exactly . do not increase or decrease the amount of medicine without your doctor ' s approval . - hard capsules should not be given to children under approximately six years of age because they may not be able to swallow them and they may 

In [36]:
leaflets_inter_content_plan[0]

"cystagon￨Section3￨PRODUCT_NAME￨AWAY cystagon￨Section3￨GENERIC_NAME￨AWAY 12￨Section3￨AGE￨AWAY 1.30￨Section3￨NUMBER￨AWAY 12￨Section3￨NUMBER￨HOME 50￨Section3￨NUMBER￨AWAY 1.95￨Section3￨NUMBER￨HOME cystagon￨Section3￨GENERIC_NAME￨HOME mouth￨Section3￨SYSTEM_ORGAN_SITE￨AWAY cystagon￨Section3￨GENERIC_NAME￨HOME hard_capsules￨Section3￨TREATMENT￨AWAY starch_based_foods￨Section3￨TREATMENT￨AWAY your_child's_medical_treatment￨Section3￨TREATMENT￨AWAY cystagon￨Section3￨GENERIC_NAME￨HOME important_electrolytes￨Section3￨TEST￨AWAY kidneys￨Section3￨SYSTEM_ORGAN_SITE￨AWAY these_supplements￨Section3￨TREATMENT￨HOME the_supplements￨Section3￨TREATMENT￨HOME weakness￨Section3￨DX_NAME￨HOME drowsiness￨Section3￨DX_NAME￨AWAY regular_blood_tests￨Section3￨TEST￨AWAY cystine_inside_white_blood_cells￨Section3￨TREATMENT￨AWAY cystagon￨Section3￨GENERIC_NAME￨HOME the_blood_tests￨Section3￨TEST￨HOME regular_blood_and_urine_tests￨Section3￨TEST￨HOME these_supplements￨Section3￨TREATMENT￨AWAY cystagon￨Section3￨TREATMENT_NAME￨HOME 

In [37]:
roto_pts_content = []

# for each leaflet
for leflet_num in range(len(leaflet_tgt_train)):
    
    # get current leaflet and content plan
    current_leaflet = leaflet_tgt_train[leflet_num].split()
    current_content_plan = leaflets_inter_content_plan[leflet_num].split()
    
    # get the values of content plan
    instances = []
    for entry in current_content_plan:
        record_values = entry.split(DELIM)[0]
        instances.append(record_values)
    
    # pairs (index_tgt, index_contentplan) for each leaflet
    current_str = []
    
    # for each token in current summary
    for token_pos in range(len(current_leaflet)):
        
        # get token
        token = current_leaflet[token_pos]
        
        # possible tokens if 2 words in content plan like 'immunodeficiency_syndrome'
        if token_pos < (len(current_leaflet)-1):
            token_2words = current_leaflet[token_pos] + "_" + current_leaflet[token_pos+1]
        else:
            token_2words = 'something that would never be in the section content'
        
        ### my-new-change
        # possible tokens if 3 words in content plan
        if token_pos < (len(current_leaflet)-2):
            token_3words = current_leaflet[token_pos] + "_" + current_leaflet[token_pos+1] + "_" + current_leaflet[token_pos+2]
        else:
            token_3words = 'something that would never be in the section content'
        
        # possible tokens if 4 words in content plan
        if token_pos < (len(current_leaflet)-3):
            token_4words = current_leaflet[token_pos] + "_" + current_leaflet[token_pos+1] + "_" + current_leaflet[token_pos+2] + "_" + current_leaflet[token_pos+3]
        else:
            token_4words = 'something that would never be in the section content'
        
        
        for content_plan_index in range(len(instances)):
                
            if token_4words == instances[content_plan_index]:
                # mask the corresponding position in content plan
                instances[content_plan_index] = "MASKED"
                pair = str(token_pos) + "," + str(content_plan_index)
                current_str.append(pair)

                # find just one match
                break
            
            if token_3words == instances[content_plan_index]:
                # mask the corresponding position in content plan
                instances[content_plan_index] = "MASKED"
                pair = str(token_pos) + "," + str(content_plan_index)
                current_str.append(pair)
                # find just one match
                break
            
            if token_2words == instances[content_plan_index]:
                # mask the corresponding position in content plan
                instances[content_plan_index] = "MASKED"
                pair = str(token_pos) + "," + str(content_plan_index)
                current_str.append(pair)
                # find just one match
                break
            
            if token == instances[content_plan_index]:
                # mask the corresponding position in content plan
                instances[content_plan_index] = "MASKED"
                pair = str(token_pos) + "," + str(content_plan_index)
                current_str.append(pair)
                # find just one match
                break
    
    # join pairs into string with " " between pairs
    current_str = " ".join(current_str)
    
    # add "\n" at the end
    current_str += "\n"
    
    roto_pts_content.append(current_str)

In [38]:
OUTPUT_FILE = "train-roto-ptrs.txt"

src_file = open(OUTPUT_FILE, 'w')
for src_instance in roto_pts_content:
    src_file.write(src_instance)
src_file.close()

In [39]:
len(roto_pts_content)

1044

In [40]:
index = 500

In [41]:
roto_pts_content[index]

'2,1 48,0 54,3 75,2 87,5 99,6 114,4 128,8 131,9 135,10 138,11 141,12 143,13 145,14 146,15 149,16 159,17 163,18 181,20 183,21 185,22 190,23 198,24 201,25 219,27 228,7 277,29 297,30\n'

In [42]:
len(roto_pts_content[index].split())

28

In [43]:
len(leaflets_inter_content_plan[index].split())

31

### Check whether correct

In [44]:
content_plan_indeces = []

for pair in roto_pts_content[index].split():
    pair = pair.split(",")
    
    a = int(pair[0])
    b = int(pair[1])
    
    content_plan_indeces.append(b)
    
    print(leaflet_tgt_train[index].split()[a:a+4], end=" ----> ")
    print(leaflets_inter_content_plan[index].split()[b])
    

['this', 'medicine', 'exactly', 'as'] ----> this_medicine￨Section3￨TREATMENT￨HOME
['kinzalkomb', 'with', 'or', 'without'] ----> kinzalkomb￨Section3￨PRODUCT_NAME￨AWAY
['the', 'tablets', 'should', 'be'] ----> the_tablets￨Section3￨TREATMENT￨AWAY
['kinzalkomb', 'every', 'day', 'until'] ----> kinzalkomb￨Section3￨GENERIC_NAME￨HOME
['liver', 'is', 'not', 'working'] ----> liver￨Section3￨SYSTEM_ORGAN_SITE￨HOME
['40', 'mg', '/', '12'] ----> 40￨Section3￨NUMBER￨HOME
['kinzalkomb', 'than', 'you', 'should'] ----> kinzalkomb￨Section3￨GENERIC_NAME￨HOME
['symptoms', 'such', 'as', 'low'] ----> symptoms￨Section3￨PROBLEM￨HOME
['low', 'blood', 'pressure', 'and'] ----> low_blood_pressure￨Section3￨DX_NAME￨AWAY
['rapid', 'heartbeat', '.', 'slow'] ----> rapid_heartbeat￨Section3￨DX_NAME￨AWAY
['slow', 'heartbeat', ',', 'dizziness'] ----> slow_heartbeat￨Section3￨DX_NAME￨AWAY
['dizziness', ',', 'vomiting', ','] ----> dizziness￨Section3￨DX_NAME￨AWAY
['vomiting', ',', 'reduced', 'kidney'] ----> vomiting￨Section3￨DX_

In [45]:
# check out pairs missed
for i in range(0, len(leaflets_inter_content_plan[index].split()), 1):
    if i not in content_plan_indeces:
        print(leaflets_inter_content_plan[index].split()[i])
        
# explanation - 3-word long token
# explanation - hiv_infection ---> bc NER outputs - 'hiv', 'hiv_infection' - in content plan I have 2 tokens starting with 'hiv'

low_blood_levels_of_potassium￨Section3￨DX_NAME￨AWAY
certain_anti-arrhythmic_treatments￨Section3￨TREATMENT￨HOME
kinzalkomb￨Section3￨GENERIC_NAME￨AWAY


In [90]:
leaflet_tgt_train[500]

"zyclara 3 . 75 % cream contains the active substance imiquimod , which is an immune response modifier ( to stimulate the human immune system ). this medicine is prescribed for the treatment of actinic keratosis in adults . this medicine stimulates your body ' s own immune system to produce natural substances which help fight your actinic keratosis . actinic keratosis appears as rough areas of skin found in people who have been exposed to a lot of sunshine over the course of their lifetime . these areas can be the same colour as your skin or are greyish , pink , red or brown . they can be flat and scaly , or raised , rough , hard and warty . this medicine should only be used for actinic keratosis on the face or scalp if your doctor has decided that it is the most appropriate treatment for you .\n"

In [91]:
leaflets_inter_content_plan[500]

'zyclara￨Section1￨PRODUCT_NAME￨HOME zyclara￨Section1￨BRAND_NAME￨HOME 3.75￨Section1￨NUMBER￨HOME the_active_substance_imiquimod￨Section1￨TREATMENT￨AWAY an_immune_response_modifier￨Section1￨TREATMENT￨AWAY this_medicine￨Section1￨TREATMENT￨AWAY actinic_keratosis￨Section1￨DX_NAME￨HOME this_medicine￨Section1￨TREATMENT￨AWAY natural_substances￨Section1￨TREATMENT￨AWAY your_actinic_keratosis￨Section1￨PROBLEM￨HOME actinic_keratosis￨Section1￨DX_NAME￨HOME skin￨Section1￨SYSTEM_ORGAN_SITE￨HOME skin￨Section1￨SYSTEM_ORGAN_SITE￨HOME greyish￨Section1￨DX_NAME￨AWAY pink￨Section1￨DX_NAME￨HOME red￨Section1￨DX_NAME￨HOME flat￨Section1￨DX_NAME￨AWAY scaly￨Section1￨DX_NAME￨HOME warty￨Section1￨DX_NAME￨HOME this_medicine￨Section1￨TREATMENT￨AWAY actinic_keratosis￨Section1￨DX_NAME￨AWAY face￨Section1￨SYSTEM_ORGAN_SITE￨AWAY scalp￨Section1￨SYSTEM_ORGAN_SITE￨HOME \n'